In [ ]:
!pip install music21
!pip install mido
!pip install pretty_midi

In [ ]:
import pretty_midi
from music21 import *
import numpy as np
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import time
import os
import glob
from itertools import groupby
import math
import random
import pickle
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, GlobalMaxPooling1D, Activation, GlobalMaxPooling2D
from keras_self_attention import SeqSelfAttention
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.layers import Layer
from tensorflow.python.client import device_lib
import keras
import gc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import re
import string
import random
from keras.layers import TFSMLayer
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit([np.arange(128).tolist()])

encoded_data_path = "/content/drive/MyDrive/encoded_doug_mckenzie_midi_16_v2/encoded_doug_mckenzie_midi_16_v2/"

output_path = "./output/"

batch_size = 32
sequence_length = 300
generate_sample_every_ep = 100

maxlen = sequence_length  # Max sequence size
embed_dim = 128  # Embedding size for each token
num_heads = 4  # Number of attention heads
feed_forward_dim = 128  # Hidden layer size in feed forward network inside transformer

combi_to_int_pickle = "combi_to_int.pickle"
int_to_combi_pickle = "int_to_combi.pickle"
vocab_pickle = "vocab.pickle"

vocab_size = 40000
unk_tag_str = '<UNK>'
unk_tag_idx = 0
pad_tag_str = ''
pad_tag_idx = 1

In [ ]:

with open('/content/drive/MyDrive/model_gpt_3_1_lightweight/combi_to_int.pickle', 'rb') as f:
    combi_to_int = pickle.load(f)

with open('/content/drive/MyDrive/all_song_tokenised.pickle', 'rb') as f:
    all_song_tokenised = pickle.load(f)

with open('/content/drive/MyDrive/model_gpt_3_1_lightweight/int_to_combi.pickle', 'rb') as f:
    int_to_combi = pickle.load(f)

In [ ]:
model = tf.saved_model.load('/content/drive/MyDrive/model_gpt_3_1_lightweight')
infer = model.signatures["serving_default"]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
seed_len = 300
num_note_to_gen = 1000

# song_idx = random.randint(0,len(all_song_tokenised)-1)
song_idx = 8
print("Song index: ", song_idx)
seq_start_at = random.randint(0,len(all_song_tokenised[song_idx])-sequence_length)
print("seq_start_at: ", seq_start_at)
start_tokens = all_song_tokenised[song_idx][seq_start_at:seq_start_at + seed_len].tolist()
print("Start tokens: ", start_tokens)
while (start_tokens == [()]*sequence_length):
    print("Got all zeros, rerolling")
    song_idx = random.randint(0,len(all_song_tokenised)-1)
    seq_start_at = random.randint(0,len(all_song_tokenised[song_idx])-sequence_length)
    start_tokens = all_song_tokenised[song_idx][seq_start_at:seq_start_at + sequence_length].tolist()
print("Start tokens: ", start_tokens)
print(len(start_tokens))
ori = start_tokens.copy()
backup = ori.copy()


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def sample_from(logits, k):
    logits, indices = tf.math.top_k(logits, k= k, sorted=True)
    indices = np.asarray(indices).astype("int32")
    preds = np.asarray(logits).astype("float32")
    if(unk_tag_idx in indices):
        unk_tag_position = np.where(indices == unk_tag_idx)[0].item()
        indices = np.delete(indices, unk_tag_position)
        preds = np.delete(preds, unk_tag_position)
    preds = softmax(preds)
    return np.random.choice(indices, p=preds)

def convertToRoll(seq_list):
    seq_list = [int_to_combi[i] for i in seq_list]
    roll = mlb.transform(seq_list)
    print(seq_list)
    return roll


k = 10
tokens_generated = []
num_tokens_generated = 0

while num_tokens_generated <= num_note_to_gen:

    x = start_tokens[-sequence_length:]
    print(x)
    pad_len = maxlen - len(start_tokens)
    print(pad_len)
    sample_index = -1
    if pad_len > 0:
        x = start_tokens + [0] * pad_len
        sample_index = len(start_tokens) - 1

    x = np.array([x])
    print("x shape: ", x.shape)
    x = tf.convert_to_tensor(x, dtype=tf.int32)

    y = infer(x)['dense_56']
    sample_token = sample_from(y[0][sample_index], 10)
    tokens_generated.append(sample_token)
    start_tokens.append(sample_token)
    num_tokens_generated = len(tokens_generated)
    print(f"generated {num_tokens_generated} notes")

piano_roll = convertToRoll(start_tokens)
print("-------------------------------------------")
ori = convertToRoll(ori)


In [ ]:
def piano_roll_to_pretty_midi(piano_roll_in, fs, program=0, velocity = 64):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    piano_roll = np.where(piano_roll_in == 1, 64, 0)
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI(initial_tempo=100.0)
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')
    print(piano_roll.shape)

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm


In [ ]:
bpm = 125
fs = 1/((60/bpm)/4)
name = song_idx
# fs = 100
# ori = np.array(ori)
mid_out = piano_roll_to_pretty_midi(piano_roll.T, fs=fs)
mid_ori = piano_roll_to_pretty_midi(ori.T, fs=fs)
midi_out_path = output_path+f"gpt-v3-id-{name}.mid"
if midi_out_path is not None:
        mid_out.write(midi_out_path)

midi_ori_path = output_path+f"ori-gpt-v3-id-{name}.mid"
if midi_ori_path is not None:
        mid_ori.write(midi_ori_path)